In [1]:
import opensim as osim
import numpy as np
import hyperobjects as ho
import common
import clifford as cf
from clifford import g3c
import clifford.tools.g3c as g3c_tools
from opensim import Vec3
from enum import Enum

G3c, blades_g3c, stuff = g3c.layout, g3c.blades, g3c.stuff

e1, e2, e3 = g3c.e1, g3c.e2, g3c.e3
e12, e13, e23 = g3c.e12, g3c.e13, g3c.e23
e31 = -e13
Px, Py, Pz = g3c.e23, -g3c.e13, g3c.e12
up, down, homo = g3c.up, g3c.down, g3c.homo
no, ni = g3c.eo, g3c.einf
E0 = g3c.E0
I_base = g3c.I_base

c:\Users\glaisne\mambaforge\envs\workspace\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
c:\Users\glaisne\mambaforge\envs\workspace\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
c:\Users\glaisne\mambaforge\envs\workspace\Lib\site-packages\pyganja\__init__.py:2: UserWarning: Failed to import cef_gui, cef functions will be unavailable
  from .script_api import *
c:\Users\glaisne\mambaforge\envs\workspace\Lib\

In [2]:
class MultiVectorType(Enum):
    NotBlade = -1
    Vector = 0
    EuclideanPoint = 1
    ConformalPoint = 2
    PointPair = 3
    Circle = 4
    Line = 5
    Sphere = 6
    Plane = 7

def blade_type(mv):
    if not isinstance(mv, cf.MultiVector):
        return MultiVectorType.NotBlade
    
    obj = g3c_tools.interpret_multivector_as_object(mv)
    if obj == -1:
        return MultiVectorType.NotBlade
    elif obj == 0:
        return MultiVectorType.Vector
    elif obj == 1:
        return MultiVectorType.EuclideanPoint
    elif obj == 2:
        return MultiVectorType.ConformalPoint
    elif obj == 3:
        return MultiVectorType.PointPair
    elif obj == 4:
        return MultiVectorType.Circle
    elif obj == 5:
        return MultiVectorType.Line
    elif obj == 6:
        return MultiVectorType.Sphere
    elif obj == 7:
        return MultiVectorType.Plane
    
def to_vector(v):
    if isinstance(v, osim.Vec3):
        v = v.to_numpy()
    return v[0]*e1 + v[1]*e2 + v[2]*e3

def to_point(p):
    return up(to_vector(p))

In [9]:
n = e1 + e2 + e3
p = 4*e1
L = g3c_tools.random_line()
pla = n + (p | n)*ni
blade_type(L)
blade_type(L.dual())

<MultiVectorType.PointPair: 3>

In [3]:
def to_array(mv):
    b_type = blade_type(mv)
    if b_type == MultiVectorType.Vector:
        return np.array([mv | e1, mv | e2, mv | e3]).astype(float)
    elif b_type == MultiVectorType.EuclideanPoint:
        return np.array([mv | e1, mv | e2, mv | e3]).astype(float)
    elif b_type == MultiVectorType.ConformalPoint:
        mv = down(mv)
        return np.array([mv | e1, mv | e2, mv | e3]).astype(float)
    elif b_type == MultiVectorType.Line:
        # return position then direction
        P, D = g3c_tools.line_to_point_and_direction(mv)
        if P == 0:
            return [np.array([0,0,0]), to_array(D)]
        return [to_array(P), to_array(D)]
    elif b_type == MultiVectorType.Plane:
        # return orthogonal proj of origin onto it THEN normal
        location = g3c_tools.get_nearest_plane_point(mv)
        normal = g3c_tools.get_plane_normal(mv)
        if location == 0:
            return [np.array([0,0,0]), to_array(normal)]
        return [to_array(location), to_array(normal)]
    else:
        raise TypeError(f"Can't convert {b_type} as numpy array.")

def rotation_axis_in_joint_frame(model, Q, last_frame):
    W_in_ground = common.frame_jacobian(model, Q, last_frame)
    W_in_joints = []
    all_joints = [j for j in model.getJointSet()]
    for i, w_in_ground in enumerate(W_in_ground.T):
        Ji = all_joints[i // 3]
        w_in_joint = model.getGround().expressVectorInAnotherFrame(Q, Vec3(*w_in_ground), Ji.getParentFrame()).to_numpy()
        W_in_joints.append(w_in_joint)
    return np.array(W_in_joints).T

def lever_arms(model, Q, W, origin, frame_origin, insertion, frame_insertion):
    if blade_type(origin) == MultiVectorType.NotBlade:
        if not isinstance(origin, osim.Vec3):
            origin = osim.Vec3(*origin)
    else:
        origin = osim.Vec3(*to_array(origin))

    if blade_type(insertion) == MultiVectorType.NotBlade:
        if not isinstance(insertion, osim.Vec3):
            insertion = osim.Vec3(*insertion)
    else:
        insertion = osim.Vec3(*to_array(insertion))

    lever_arms = []
    for i, c in enumerate(model.getCoordinateSet()):
        # c: osim.Coordinate = c
        FJ: osim.Joint = c.getJoint().getParentFrame()
        O_in_joint = frame_origin.findStationLocationInAnotherFrame(Q, origin, FJ).to_numpy()
        I_in_joint = frame_insertion.findStationLocationInAnotherFrame(Q, insertion, FJ).to_numpy()
        moment = np.cross(I_in_joint, O_in_joint) / np.linalg.norm(I_in_joint - O_in_joint)
        lever_arm = W[:, i] @ moment
        lever_arms.append(lever_arm)
    
    return np.array(lever_arms)

In [19]:
X = g3c_tools.random_conformal_point()

model = osim.Model()
ground: osim.Ground = model.getGround()

np.random.seed(28)

k = 5
B = [osim.Body(f"B{i}", 1.0, Vec3(0), osim.Inertia(0)) for i in range(1, k+1)]
B[0] = model.getGround()
J = [osim.BallJoint(f"J{i}", B[i-1], Vec3(*np.random.uniform(-1,1,3)), Vec3(*np.random.uniform(-np.pi,np.pi,3)), B[i], Vec3(*np.
random.uniform(-1,1,3)), Vec3(*np.random.uniform(-np.pi,np.pi,3))) for i in range(1, k)]

for Bi in B[1:]: model.addBody(Bi)
for Ji in J: model.addJoint(Ji)

q1 = np.random.uniform(-np.pi, np.pi, 3*(k-1))
q2 = np.random.uniform(-np.pi, np.pi, 3*(k-1))
q3 = np.random.uniform(-np.pi, np.pi, 3*(k-1))
q4 = np.random.uniform(-np.pi, np.pi, 3*(k-1))
q5 = np.random.uniform(-np.pi, np.pi, 3*(k-1))
q6 = np.random.uniform(-np.pi, np.pi, 3*(k-1))

muscle = osim.Millard2012EquilibriumMuscle("muscle", 200.0, 0.6, 0.55, 0.0) 
origin = osim.Vec3(*np.random.uniform(-1,1,3))
B_origin = B[0]
muscle.addNewPathPoint("O", B_origin, origin)
insertion = osim.Vec3(np.random.uniform(-1,1,3))
B_insertion = B[-1]
muscle.addNewPathPoint("I", B_insertion, insertion)
model.addForce(muscle)

Q_null = model.initSystem()
model.equilibrateMuscles(Q_null)

# X = Vec3(1, 2 ,3)
Q1 = osim.State(Q_null)
model.realizePosition(Q1)
model.equilibrateMuscles(Q1)
for i, c in enumerate(model.getCoordinateSet()):
    c.setValue(Q1, q1[i])
    
Q2 = osim.State(Q_null)
model.realizePosition(Q2)
model.equilibrateMuscles(Q2)
for i, c in enumerate(model.getCoordinateSet()):
    c.setValue(Q2, q2[i])
    
Q3 = osim.State(Q_null)
model.realizePosition(Q3)
model.equilibrateMuscles(Q3)
for i, c in enumerate(model.getCoordinateSet()):
    c.setValue(Q3, q3[i])
    
Q4 = osim.State(Q_null)
model.realizePosition(Q4)
model.equilibrateMuscles(Q4)
for i, c in enumerate(model.getCoordinateSet()):
    c.setValue(Q4, q4[i])
    
Q5 = osim.State(Q_null)
model.realizePosition(Q5)
model.equilibrateMuscles(Q5)
for i, c in enumerate(model.getCoordinateSet()):
    c.setValue(Q5, q5[i])

Q6 = osim.State(Q_null)
model.realizePosition(Q6)
model.equilibrateMuscles(Q6)
for i, c in enumerate(model.getCoordinateSet()):
    c.setValue(Q6, q6[i])

In [20]:
# Compute the rotation axes expressed in the ground
# NB: in any joint configuration Q, the rotation axes IN their respective joints are the same! But we can consider a case where they are not (for instance if the axis is parametrized by another function)
omegas_Q1 = common.frame_jacobian(model, Q1, B_insertion)
omegas_Q2 = common.frame_jacobian(model, Q2, B_insertion)

# Compute the rotation axes expressed in there parent frame
omegas_in_joints_Q1 = []
for i, w in enumerate(omegas_Q1.T):
    Ji = J[i // 3]
    w_in_J_Q1 = ground.expressVectorInAnotherFrame(Q1, Vec3(*w), Ji.getParentFrame()).to_numpy()
    omegas_in_joints_Q1.append(w_in_J_Q1)
omegas_in_joints_Q1 = np.array(omegas_in_joints_Q1).T

omegas_in_joints_Q2 = []
for i, w in enumerate(omegas_Q2.T):
    Ji = J[i // 3]
    w_in_J_Q2 = ground.expressVectorInAnotherFrame(Q2, Vec3(*w), Ji.getParentFrame()).to_numpy()
    omegas_in_joints_Q2.append(w_in_J_Q2)
omegas_in_joints_Q2 = np.array(omegas_in_joints_Q2).T

# Both matrices should be the same
print(omegas_in_joints_Q1.round(2))
print(omegas_in_joints_Q2.round(2))

[[ 1. -0. -0.  1. -0. -0.  1.  0.  0.  1.  0. -0.]
 [-0.  1.  0. -0.  1.  0.  0.  1. -0.  0.  1. -0.]
 [-0.  0.  1. -0.  0.  1.  0. -0.  1. -0. -0.  1.]]
[[ 1. -0. -0.  1. -0.  0.  1. -0.  0.  1. -0.  0.]
 [-0.  1.  0. -0.  1.  0. -0.  1. -0. -0.  1.  0.]
 [-0.  0.  1.  0.  0.  1.  0. -0.  1.  0.  0.  1.]]


In [21]:
W = rotation_axis_in_joint_frame(model, Q1, B_insertion)
W.round(5)

array([[ 1., -0., -0.,  1., -0., -0.,  1.,  0.,  0.,  1.,  0., -0.],
       [-0.,  1.,  0., -0.,  1.,  0.,  0.,  1., -0.,  0.,  1., -0.],
       [-0.,  0.,  1., -0.,  0.,  1.,  0., -0.,  1., -0., -0.,  1.]])

In [22]:
coordinates = model.getCoordinateSet()
lever_arms_by_derivatives_Q1 = common.lever_arm_matrix(Q1, [muscle], coordinates)
lever_arms_by_derivatives_Q2 = common.lever_arm_matrix(Q2, [muscle], coordinates)
lever_arms_by_derivatives_Q3 = common.lever_arm_matrix(Q3, [muscle], coordinates)
lever_arms_by_derivatives_Q4 = common.lever_arm_matrix(Q4, [muscle], coordinates)
lever_arms_by_derivatives_Q5 = common.lever_arm_matrix(Q5, [muscle], coordinates)
lever_arms_by_derivatives_Q6 = common.lever_arm_matrix(Q6, [muscle], coordinates)
lever_arms_Q1 = lever_arms(model, Q1, W, origin, B_origin, insertion, B_insertion)
lever_arms_Q2 = lever_arms(model, Q2, W, origin, B_origin, insertion, B_insertion)
lever_arms_Q3 = lever_arms(model, Q3, W, origin, B_origin, insertion, B_insertion)
lever_arms_Q4 = lever_arms(model, Q4, W, origin, B_origin, insertion, B_insertion)
lever_arms_Q5 = lever_arms(model, Q5, W, origin, B_origin, insertion, B_insertion)
lever_arms_Q6 = lever_arms(model, Q6, W, origin, B_origin, insertion, B_insertion)

print(lever_arms_Q1.round(4).tolist())
print(lever_arms_by_derivatives_Q1.ravel().round(4).tolist())

print(lever_arms_Q2.round(4).tolist())
print(lever_arms_by_derivatives_Q2.ravel().round(4).tolist())

print(lever_arms_Q3.round(4).tolist())
print(lever_arms_by_derivatives_Q3.ravel().round(4).tolist())

print(lever_arms_Q4.round(4).tolist())
print(lever_arms_by_derivatives_Q4.ravel().round(4).tolist())

print(lever_arms_Q5.round(4).tolist())
print(lever_arms_by_derivatives_Q5.ravel().round(4).tolist())

print(lever_arms_Q6.round(4).tolist())
print(lever_arms_by_derivatives_Q6.ravel().round(4).tolist())

[0.1037, -0.2348, -0.0002, 0.1532, 0.0941, 1.2609, -0.0248, -0.2056, -0.4367, 0.4354, -0.5651, 0.041]
[0.1037, -0.2348, -0.0002, 0.1532, 0.0941, 1.2609, -0.0248, -0.2056, -0.4367, 0.4354, -0.5651, 0.041]
[-0.1136, -0.0833, -0.1435, 0.072, -0.3971, -1.2449, 0.1883, 0.5488, 1.4979, -1.0847, 0.7885, 0.9096]
[-0.1136, -0.0833, -0.1435, 0.072, -0.3971, -1.2449, 0.1883, 0.5488, 1.4979, -1.0847, 0.7885, 0.9096]
[-0.1167, 0.2006, -0.0267, -0.0518, 0.6853, -0.2549, -0.5151, -0.7937, 0.0945, -1.498, -0.9128, -0.4123]
[-0.1167, 0.2006, -0.0267, -0.0518, 0.6853, -0.2549, -0.5151, -0.7937, 0.0945, -1.498, -0.9128, -0.4123]
[-0.122, 0.1677, -0.0456, 0.2486, -0.1806, 1.2004, -0.9679, 0.8716, -0.4006, 0.1639, -0.7299, 1.6003]
[-0.122, 0.1677, -0.0456, 0.2486, -0.1806, 1.2004, -0.9679, 0.8716, -0.4006, 0.1639, -0.7299, 1.6003]
[-0.197, 0.065, -0.1606, -0.0652, 0.4311, 0.2079, -0.0767, 0.311, 0.0885, 0.8454, -0.6569, 0.319]
[-0.197, 0.065, -0.1606, -0.0652, 0.4311, 0.2079, -0.0767, 0.311, 0.0885, 0.8454

In [23]:
plane_normal_Q1 = lever_arms_Q1[:3]
plane_location_Q1 = np.array([0,0,0])
plane_normal_Q2 = lever_arms_Q2[:3]
plane_location_Q2 = np.array([0,0,0])
plane_normal_Q3 = lever_arms_Q3[:3]
plane_location_Q3 = np.array([0,0,0])

# Let's replace the normal plane in each origin and insertion reference body.
# For this, the orientation of the normal is displaced, then the point of application is moved.
joint_frame = J[0].getParentFrame()

PO_normal_Q1 = joint_frame.expressVectorInAnotherFrame(Q1, Vec3(*plane_normal_Q1), B_origin)
PO_pos_Q1 = joint_frame.findStationLocationInAnotherFrame(Q1, Vec3(0), B_origin)

PO_normal_Q2 = joint_frame.expressVectorInAnotherFrame(Q2, Vec3(*plane_normal_Q2), B_origin)
PO_pos_Q2 = joint_frame.findStationLocationInAnotherFrame(Q2, Vec3(0), B_origin)

PO_normal_Q3 = joint_frame.expressVectorInAnotherFrame(Q3, Vec3(*plane_normal_Q3), B_origin)
PO_pos_Q3 = joint_frame.findStationLocationInAnotherFrame(Q3, Vec3(0), B_origin)

PI_normal_Q1 = joint_frame.expressVectorInAnotherFrame(Q1, Vec3(*plane_normal_Q1), B_insertion)
PI_pos_Q1 = joint_frame.findStationLocationInAnotherFrame(Q1, Vec3(0), B_insertion)
PI_normal_Q1 = Vec3(*(PI_normal_Q1.to_numpy() / np.linalg.norm(PI_normal_Q1.to_numpy())))

PI_normal_Q2 = joint_frame.expressVectorInAnotherFrame(Q2, Vec3(*plane_normal_Q2), B_insertion)
PI_normal_Q2 = Vec3(*(PI_normal_Q2.to_numpy() / np.linalg.norm(PI_normal_Q2.to_numpy())))
PI_pos_Q2 = joint_frame.findStationLocationInAnotherFrame(Q2, Vec3(0), B_insertion)

PI_normal_Q3 = joint_frame.expressVectorInAnotherFrame(Q3, Vec3(*plane_normal_Q3), B_insertion)
PI_normal_Q3 = Vec3(*(PI_normal_Q3.to_numpy() / np.linalg.norm(PI_normal_Q3.to_numpy())))
PI_pos_Q3 = joint_frame.findStationLocationInAnotherFrame(Q3, Vec3(0), B_insertion)

PO_Q1 = (to_vector(PO_normal_Q1) + (to_point(PO_pos_Q1) | to_vector(PO_normal_Q1)) * ni).dual()
PO_Q2 = (to_vector(PO_normal_Q2) + (to_point(PO_pos_Q2) | to_vector(PO_normal_Q2)) * ni).dual()
PO_Q3 = (to_vector(PO_normal_Q3) + (to_point(PO_pos_Q3) | to_vector(PO_normal_Q3)) * ni).dual()

PI_Q1 = (to_vector(PI_normal_Q1) + (to_point(PI_pos_Q1) | to_vector(PI_normal_Q1)) * ni).dual()
PI_Q2 = (to_vector(PI_normal_Q2) + (to_point(PI_pos_Q2) | to_vector(PI_normal_Q2)) * ni).dual()
PI_Q3 = (to_vector(PI_normal_Q3) + (to_point(PI_pos_Q3) | to_vector(PI_normal_Q3)) * ni).dual()
print(PO_Q1)

LO = PO_Q1.meet(PO_Q2)
LI = PI_Q1.meet(PI_Q2)

LO_2 = PO_Q2.meet(PO_Q3)
LI_2 = PI_Q2.meet(PI_Q3)

print(LO)
print(LI)
O = to_point(origin.to_numpy())
I = to_point(insertion.to_numpy())
print(O ^ LO)
print(I ^ LI)
print(O ^ LO_2)
print(I ^ LI_2)

print(LO.normal())
print(LO_2.normal())
print(LI.normal())
print(LI_2.normal())

# PO = LO.meet(LO_2)
# print(PO.normal())
# print(LO.normal())
PI = LI.meet(LI_2)
print(PI)

print(blade_type(PI))
# print(down(PI.dual()))



-(0.07345^e1234) - (0.07345^e1235) - (0.04239^e1245) - (0.25203^e1345) + (0.02355^e2345)
-(0.00443^e124) - (0.00443^e125) + (0.04463^e134) + (0.04463^e135) + (0.04098^e145) + (0.00467^e234) + (0.00467^e235) + (0.00127^e245) + (0.03034^e345)
(0.65658^e124) + (0.65658^e125) + (0.02276^e134) + (0.02276^e135) + (0.61317^e145) - (0.01555^e234) - (0.01555^e235) - (0.72089^e245) - (0.01047^e345)
0
0
0
0
-(0.08696^e124) - (0.08696^e125) + (0.87509^e134) + (0.87509^e135) + (0.80342^e145) + (0.09161^e234) + (0.09161^e235) + (0.02499^e245) + (0.59489^e345)
-(0.08696^e124) - (0.08696^e125) + (0.87509^e134) + (0.87509^e135) + (0.80342^e145) + (0.09161^e234) + (0.09161^e235) + (0.02499^e245) + (0.59489^e345)
(0.69373^e124) + (0.69373^e125) + (0.02405^e134) + (0.02405^e135) + (0.64786^e145) - (0.01642^e234) - (0.01642^e235) - (0.76168^e245) - (0.01106^e345)
-(0.51641^e124) - (0.51641^e125) + (0.04141^e134) + (0.04141^e135) - (0.40862^e145) + (0.25698^e234) + (0.25698^e235) + (0.87086^e245) - (0.27318

In [24]:
to_point(PI_pos_Q1) ^ PI_Q2

(0.63515^e12345)

In [25]:
joint_center_in_ground_Q1 = to_point(joint_frame.findStationLocationInGround(Q1, Vec3(0)))
joint_center_in_ground_Q2 = to_point(joint_frame.findStationLocationInGround(Q1, Vec3(0)))
normal_in_ground_Q1 = to_vector(joint_frame.expressVectorInGround(Q1, Vec3(*plane_normal_Q1)))
normal_in_ground_Q2 = to_vector(joint_frame.expressVectorInGround(Q2, Vec3(*plane_normal_Q2)))

PG_Q1 = (normal_in_ground_Q1 + (joint_center_in_ground_Q1 | normal_in_ground_Q1) * ni).dual()
PG_Q2 = (normal_in_ground_Q2 + (joint_center_in_ground_Q2 | normal_in_ground_Q2) * ni).dual()
L = PG_Q1.meet(PG_Q2)
O_in_ground_Q1 = to_point(B_origin.findStationLocationInGround(Q1, origin))
O_in_ground_Q2 = to_point(B_origin.findStationLocationInGround(Q2, origin))
I_in_ground_Q1 = to_point(B_insertion.findStationLocationInGround(Q1, insertion))
I_in_ground_Q2 = to_point(B_insertion.findStationLocationInGround(Q2, insertion))
print(O_in_ground_Q1 ^ L)
print(I_in_ground_Q1 ^ L)
print(I_in_ground_Q2 ^ L)


0
-(0.03491^e1234) - (0.03491^e1235) - (0.02014^e1245) - (0.11978^e1345) + (0.01119^e2345)
-(0.05401^e1234) - (0.05401^e1235) - (0.04981^e1245) + (0.00228^e1345) + (0.03695^e2345)


In [26]:
PJ_Q1 = to_vector(plane_normal_Q1).dual()
PJ_Q2 = to_vector(plane_normal_Q2).dual()
print(PJ_Q1)
print(PJ_Q2)

LJ = PJ_Q1.meet(PJ_Q2)
print("LJ", LJ)



# pos, D = to_array(LO)
# print(pos, D)

# dirt1 = B_origin.expressVectorInAnotherFrame(Q1, Vec3(*D), joint_frame)
# # L_test = to
# print(dirt1)
# LO_test = no ^ to_vector(dirt1) ^ ni
# print(LO_test.normal())
# print(LO_test ^ LJ)

O_in_joint_Q1 = to_point(B_origin.findStationLocationInAnotherFrame(Q1, origin, joint_frame))
O_in_joint_Q2 = to_point(B_origin.findStationLocationInAnotherFrame(Q2, origin, joint_frame))
I_in_joint_Q1 = to_point(B_insertion.findStationLocationInAnotherFrame(Q1, insertion, joint_frame))
I_in_joint_Q2 = to_point(B_insertion.findStationLocationInAnotherFrame(Q2, insertion, joint_frame))
print(O_in_joint_Q1^LJ)
print(O_in_joint_Q2^LJ)
print(I_in_joint_Q1^LJ)
print(I_in_joint_Q2^LJ)

(0.00017^e1245) - (0.23478^e1345) - (0.10368^e2345)
(0.14348^e1245) - (0.08325^e1345) + (0.11355^e2345)
LJ -(0.03367^e145) - (0.01489^e245) + (0.03529^e345)
0
0
(8e-05^e1245) - (0.11158^e1345) - (0.04927^e2345)
(0.0443^e1245) - (0.0257^e1345) + (0.03506^e2345)


In [27]:
to_point(PO_pos_Q1) ^ LO

0

In [28]:
ml = PI_Q1.meet(PI_Q2.meet(PI_Q3))
print(I)
print(ml)
print(ml.normal())
mlO = PO_Q1.meet(PO_Q2.meet(PO_Q3))
print(mlO.normal())
print(O)
print(I ^ PI_Q1)
print(I ^ PI_Q2)
print(I ^ PI_Q3)
print(blade_type(I))
Lk = PI_Q1.meet(PI_Q2)
print(blade_type(Lk))
Lk2 = g3c_tools.intersect_line_and_plane_to_point(PI_Q1.meet(PI_Q2), PI_Q3)
print(Lk2)
print(I)

-(0.20196^e1) - (0.83336^e2) - (0.03367^e3) - (0.1318^e4) + (0.8682^e5)
-(0.0714^e14) - (0.0714^e15) - (0.29463^e24) - (0.29463^e25) - (0.0119^e34) - (0.0119^e35) - (0.35355^e45)
-(0.20196^e14) - (0.20196^e15) - (0.83336^e24) - (0.83336^e25) - (0.03367^e34) - (0.03367^e35) - (1.0^e45)
-(0.08696^e124) - (0.08696^e125) + (0.87509^e134) + (0.87509^e135) + (0.80342^e145) + (0.09161^e234) + (0.09161^e235) + (0.02499^e245) + (0.59489^e345)
(0.20977^e1) + (0.11476^e2) - (0.93389^e3) - (0.03534^e4) + (0.96466^e5)
0
0
0
MultiVectorType.ConformalPoint
MultiVectorType.Line
-(0.20196^e1) - (0.83336^e2) - (0.03367^e3) - (0.1318^e4) + (0.8682^e5)
-(0.20196^e1) - (0.83336^e2) - (0.03367^e3) - (0.1318^e4) + (0.8682^e5)


In [29]:
pI_interest = no ^ down(Lk2) ^ ni
pI_interest

-(0.20196^e145) - (0.83336^e245) - (0.03367^e345)

In [37]:
O_random = g3c_tools.random_conformal_point()
O_random = g3c_tools.project_points_to_line([O_random], LO)[0]
I_random = g3c_tools.random_conformal_point()
# I_random = g3c_tools.project_points_to_line([I_random], PI)[0]
I_random = (I_random | PI) / PI
# I_random = (I_random | pI_interest) / pI_interest

lever_arms_Q1_random = lever_arms(model, Q1, W, O_random, B_origin, I_random, B_insertion)
lever_arms_Q2_random = lever_arms(model, Q2, W, O_random, B_origin, I_random, B_insertion)
lever_arms_Q3_random = lever_arms(model, Q3, W, O_random, B_origin, I_random, B_insertion)
lever_arms_Q4_random = lever_arms(model, Q4, W, O_random, B_origin, I_random, B_insertion)
lever_arms_Q5_random = lever_arms(model, Q5, W, O_random, B_origin, I_random, B_insertion)
lever_arms_Q6_random = lever_arms(model, Q6, W, O_random, B_origin, I_random, B_insertion)

print(to_array(O))
print(to_array(I_random))


print((lever_arms_Q1[:3] / np.linalg.norm(lever_arms_Q1[:3])).round(4).tolist())
print((lever_arms_Q1_random[:3] / np.linalg.norm(lever_arms_Q1_random[:3])).round(4).tolist())

print((lever_arms_Q2[:3] / np.linalg.norm(lever_arms_Q2[:3])).round(4).tolist())
print((lever_arms_Q2_random[:3] / np.linalg.norm(lever_arms_Q2_random[:3])).round(4).tolist())

print((lever_arms_Q3[:3] / np.linalg.norm(lever_arms_Q3[:3])).round(4).tolist())
print((lever_arms_Q3_random[:3] / np.linalg.norm(lever_arms_Q3_random[:3])).round(4).tolist())

print((lever_arms_Q4[:3] / np.linalg.norm(lever_arms_Q4[:3])).round(4).tolist())
print((lever_arms_Q4_random[:3] / np.linalg.norm(lever_arms_Q4_random[:3])).round(4).tolist())

print((lever_arms_Q5[:3] / np.linalg.norm(lever_arms_Q5[:3])).round(4).tolist())
print((lever_arms_Q5_random[:3] / np.linalg.norm(lever_arms_Q5_random[:3])).round(4).tolist())

print((lever_arms_Q6[:3] / np.linalg.norm(lever_arms_Q6[:3])).round(4).tolist())
print((lever_arms_Q6_random[:3] / np.linalg.norm(lever_arms_Q6_random[:3])).round(4).tolist())

[ 0.20976702  0.11475728 -0.93389042]
[-0.20195964 -0.83335775 -0.03366729]
[0.404, -0.9148, -0.0006]
[0.404, -0.9148, -0.0006]
[-0.5649, -0.4141, -0.7137]
[-0.5649, -0.4141, -0.7137]
[-0.4995, 0.8588, -0.1141]
[-0.4995, 0.8588, -0.1141]
[-0.5744, 0.7899, -0.2146]
[-0.5744, 0.7899, -0.2146]
[-0.751, 0.2479, -0.6119]
[-0.751, 0.2479, -0.6119]
[0.5578, -0.8077, 0.1913]
[0.5578, -0.8077, 0.1913]


In [51]:
pointI = to_point(B_insertion.findStationLocationInAnotherFrame(Q1, Vec3(*to_array(I_random)), ground))
pointO = to_point(B_origin.findStationLocationInAnotherFrame(Q1, origin, ground))
LOI = pointO^pointI^ni
I_random2 = g3c_tools.random_conformal_point()
I_random2 = (I_random2 | LOI) / LOI
print(I_random2 ^ LOI)
pointI2 = to_point(ground.findStationLocationInAnotherFrame(Q1, Vec3(*to_array(I_random2)), B_insertion))

I_random_in_ground = to_point(B_insertion.findStationLocationInAnotherFrame(Q1, Vec3(*to_array(I_random)), ground))
print(O^LOI)
print(I_random_in_ground^LOI)
print(I_random2^LOI)

lever_arms_Q1_random = lever_arms(model, Q1, W, O_random, B_origin, pointI2, B_insertion)
lever_arms_Q2_random = lever_arms(model, Q2, W, O_random, B_origin, pointI2, B_insertion)
lever_arms_Q3_random = lever_arms(model, Q3, W, O_random, B_origin, pointI2, B_insertion)
lever_arms_Q4_random = lever_arms(model, Q4, W, O_random, B_origin, pointI2, B_insertion)
lever_arms_Q5_random = lever_arms(model, Q5, W, O_random, B_origin, pointI2, B_insertion)
lever_arms_Q6_random = lever_arms(model, Q6, W, O_random, B_origin, pointI2, B_insertion)

print((lever_arms_Q1[:3] / np.linalg.norm(lever_arms_Q1[:3])).round(4).tolist())
print((lever_arms_Q1_random[:3] / np.linalg.norm(lever_arms_Q1_random[:3])).round(4).tolist())

print((lever_arms_Q2[:3] / np.linalg.norm(lever_arms_Q2[:3])).round(4).tolist())
print((lever_arms_Q2_random[:3] / np.linalg.norm(lever_arms_Q2_random[:3])).round(4).tolist())

print((lever_arms_Q3[:3] / np.linalg.norm(lever_arms_Q3[:3])).round(4).tolist())
print((lever_arms_Q3_random[:3] / np.linalg.norm(lever_arms_Q3_random[:3])).round(4).tolist())

print((lever_arms_Q4[:3] / np.linalg.norm(lever_arms_Q4[:3])).round(4).tolist())
print((lever_arms_Q4_random[:3] / np.linalg.norm(lever_arms_Q4_random[:3])).round(4).tolist())

print((lever_arms_Q5[:3] / np.linalg.norm(lever_arms_Q5[:3])).round(4).tolist())
print((lever_arms_Q5_random[:3] / np.linalg.norm(lever_arms_Q5_random[:3])).round(4).tolist())

print((lever_arms_Q6[:3] / np.linalg.norm(lever_arms_Q6[:3])).round(4).tolist())
print((lever_arms_Q6_random[:3] / np.linalg.norm(lever_arms_Q6_random[:3])).round(4).tolist())

0
0
0
0
[0.404, -0.9148, -0.0006]
[0.404, -0.9148, -0.0006]
[-0.5649, -0.4141, -0.7137]
[-0.4918, -0.5282, -0.6922]
[-0.4995, 0.8588, -0.1141]
[-0.6489, 0.6857, -0.3297]
[-0.5744, 0.7899, -0.2146]
[-0.6106, -0.3278, -0.7209]
[-0.751, 0.2479, -0.6119]
[-0.6106, 0.7452, -0.268]
[0.5578, -0.8077, 0.1913]
[0.3071, -0.9457, -0.1061]


In [58]:
I_random^down(I_random2)^ni
# I_random2

-(1.0^e45)

In [57]:
no^down(I_random2)^ni

-(0.56824^e145) + (3.31411^e245) + (17.6572^e345)

In [274]:
print(PI.normal())
print(((PI.normal()|(no))^ni)|no)

-(0.7911^e14) - (0.7911^e15) - (0.47482^e24) - (0.47482^e25) - (0.7111^e34) - (0.7111^e35) + (1.0^e45)
-(0.7911^e1) - (0.47482^e2) - (0.7111^e3) + (0.5^e4) - (0.5^e5)


In [275]:

print(lever_arms_by_derivatives_Q1.ravel().round(4).tolist())
print(lever_arms_by_derivatives_Q2.ravel().round(4).tolist())
lever_arms_Q1 = []
lever_arms_Q2 = []
for i, c in enumerate(model.getCoordinateSet()):
    # c: osim.Coordinate = c
    FJ: osim.Joint = c.getJoint().getParentFrame()
    origin_in_J_Q1 = B_origin.findStationLocationInAnotherFrame(Q1, O_random, FJ).to_numpy()
    insertion_in_J_Q1 = B_insertion.findStationLocationInAnotherFrame(Q1, I_random, FJ).to_numpy()
    la = np.cross(insertion_in_J_Q1, origin_in_J_Q1) / np.linalg.norm(insertion_in_J_Q1 - origin_in_J_Q1)
    w_Q1 = omegas_in_joints_Q1[:, i]
    la = w_Q1 @ la
    lever_arms_Q1.append(la)

    origin_in_J_Q2 = B_origin.findStationLocationInAnotherFrame(Q2, O_random, FJ).to_numpy()
    insertion_in_J_Q2 = B_insertion.findStationLocationInAnotherFrame(Q2, I_random, FJ).to_numpy()

    la = np.cross(insertion_in_J_Q2, origin_in_J_Q2) / np.linalg.norm(insertion_in_J_Q2 - origin_in_J_Q2)
    w_Q2 = omegas_in_joints_Q2[:, i]
    la = w_Q2 @ la
    lever_arms_Q2.append(la)

lever_arms_Q1 = np.array(lever_arms_Q1)
lever_arms_Q2 = np.array(lever_arms_Q2)

# print(lever_arms_Q1.round(4).tolist())
print(lever_arms_Q1[:3].round(4).tolist() / np.linalg.norm(lever_arms_Q1[:3].round(4).tolist()))
print(lever_arms_by_derivatives_Q1.ravel()[:3].round(4).tolist() / np.linalg.norm(lever_arms_by_derivatives_Q1.ravel()[:3]))

# print(lever_arms_Q2.round(4).tolist())
# print(lever_arms_by_derivatives_Q2.ravel().round(4).tolist())
print(lever_arms_Q2[:3].round(4).tolist() / np.linalg.norm(lever_arms_Q2[:3].round(4).tolist()))
print(lever_arms_by_derivatives_Q2.ravel()[:3].round(4).tolist() / np.linalg.norm(lever_arms_by_derivatives_Q2.ravel()[:3]))

[0.7256, -0.7664, 0.9272, 1.1941, 1.1508, 0.2053, -0.675, -0.5937, -0.2545, -0.0895, -0.3261, 0.3516]
[0.6668, -0.5551, 0.3662, 0.252, -1.1602, -0.2614, 0.6664, 0.6613, 0.8102, -0.1866, -0.2008, 0.6752]


TypeError: in method 'Frame_findStationLocationInAnotherFrame', argument 3 of type 'SimTK::Vec3 const &'

In [12]:

# n_I_q1 = J[0].getParentFrame().expressVectorInAnotherFrame(Q1, Vec3(*n1), B_origin).to_numpy()
# Otest = J[0].getParentFrame().findStationLocationInAnotherFrame(Q1, Vec3(0), B_origin).to_numpy()

print(S ^ up(to_multivec(O_in_ground + 0.1*L1_dir_in_ground)))
print(S ^ up(to_multivec(I_in_ground + 0.1*L2_dir_in_ground)))
print(S ^ up(to_multivec(I_in_ground - 50*L2_dir_in_ground)))

k1 = (g3c_tools.random_conformal_point() | S) / S
k2 = (g3c_tools.random_conformal_point() | S) / S
print(k1)
print(S ^ k1)
print(S ^ k2)
kO = point_to_np(k1)
kI = point_to_np(k2)
kO_in = Vec3(*ground.findStationLocationInAnotherFrame(Q1, Vec3(*kO), B_origin).to_numpy())
kI_in = Vec3(*ground.findStationLocationInAnotherFrame(Q1, Vec3(*kI), B_insertion).to_numpy())

kO2_in = Vec3(*ground.findStationLocationInAnotherFrame(Q2, Vec3(*kO), B_origin).to_numpy())
kI2_in = Vec3(*ground.findStationLocationInAnotherFrame(Q2, Vec3(*kI), B_insertion).to_numpy())

lever_arms_Q1 = []
lever_arms_Q2 = []
for i, c in enumerate(model.getCoordinateSet()):
    # c: osim.Coordinate = c
    FJ: osim.Joint = c.getJoint().getParentFrame()
    origin_in_J_Q1 = B_origin.findStationLocationInAnotherFrame(Q1, kO_in, FJ).to_numpy()
    insertion_in_J_Q1 = B_insertion.findStationLocationInAnotherFrame(Q1, kI_in, FJ).to_numpy()
    la = np.cross(insertion_in_J_Q1, origin_in_J_Q1) / np.linalg.norm(insertion_in_J_Q1 - origin_in_J_Q1)
    w_Q1 = omegas_in_joints_Q1[:, i]
    la = w_Q1 @ la
    lever_arms_Q1.append(la)

    origin_in_J_Q2 = B_origin.findStationLocationInAnotherFrame(Q2, Vec3(*kO), FJ).to_numpy()
    insertion_in_J_Q2 = B_insertion.findStationLocationInAnotherFrame(Q2, Vec3(*kI), FJ).to_numpy()
    la = np.cross(insertion_in_J_Q2, origin_in_J_Q2) / np.linalg.norm(insertion_in_J_Q2 - origin_in_J_Q2)
    w_Q2 = omegas_in_joints_Q2[:, i]
    la = w_Q2 @ la
    lever_arms_Q2.append(la)

lever_arms_Q1 = np.array(lever_arms_Q1)
lever_arms_Q2 = np.array(lever_arms_Q2)


la_by_derivatives_Q1 = common.lever_arm_matrix(Q1, [muscle], model.getCoordinateSet())
la_by_derivatives_Q2 = common.lever_arm_matrix(Q2, [muscle], model.getCoordinateSet())

print(lever_arms_Q1.round(4).tolist())
print(la_by_derivatives_Q1.ravel().round(4).tolist())

print((lever_arms_Q1[:3] / np.linalg.norm(lever_arms_Q1[:3])).round(4).tolist())
print((la_by_derivatives_Q1[:3] / np.linalg.norm(la_by_derivatives_Q1[:3])).ravel().round(4).tolist())

print("OK")
print(lever_arms_Q2.round(4).tolist())
print(la_by_derivatives_Q2.ravel().round(4).tolist())
print((lever_arms_Q2[:3] / np.linalg.norm(lever_arms_Q2[:3])).round(4).tolist())
print((la_by_derivatives_Q2[:3] / np.linalg.norm(la_by_derivatives_Q2[:3])).ravel().round(4).tolist())

0
0
0
-(1.35764^e1) + (7.06332^e2) + (4.74274^e3) + (58.29926^e4) + (59.29926^e5)
0
0
[1.0354, -0.8731, -3.0674, -2.5531, 1.3454, -0.7172, -1.0005, 3.1802, -1.954, 3.6992, -1.145, -1.3789]
[0.0706, -0.0596, -0.2092, -0.0854, 0.3853, 1.3011, -0.2036, -0.0042, -0.7931, 0.68, -0.3989, -0.1945]
[0.3088, -0.2604, -0.9148]
[0.3088, -0.2604, -0.9148]
OK
[-0.5595, -0.0589, 1.2261, -0.6369, -0.0773, 0.5306, -0.0176, 0.8053, -0.9171, 1.2382, -1.8551, 0.4389]
[-0.1876, -0.0005, -0.1596, 0.0515, -0.6161, -0.7833, 0.1244, 0.4488, 1.3303, -0.6104, 0.6434, 1.0534]
[-0.4147, -0.0437, 0.9089]
[-0.7617, -0.0022, -0.648]


In [154]:
%matplotlib qt5

import matplotlib.pyplot as plt
import itertools

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection="3d")

all_O_test = []
all_I_test = []
alphas = np.linspace(-0.1,0.1,20)
for a in alphas:
    O_test = (origin.to_numpy() + a*direction_vec)
    O_test = B_origin.expressVectorInGround(Q1, Vec3(*O_test)).to_numpy()
    all_O_test.append(O_test)

    I_test = (insertion.to_numpy() + a*direction_vec2)
    I_test = B_insertion.expressVectorInGround(Q1, Vec3(*I_test)).to_numpy()
    all_I_test.append(I_test)
    
permut = itertools.permutations(list(range(len(alphas))), 2)
for permu in permut:
    i1, i2 = permu
    ax.plot([all_O_test[i1][0], all_I_test[i2][0]],[all_O_test[i1][1], all_I_test[i2][1]],[all_O_test[i1][2], all_I_test[i2][2]])#, c="red" )

# ax.plot([O_in_ground[0], I_in_ground[0]],[O_in_ground[1], I_in_ground[1]],[O_in_ground[2], I_in_ground[2]], c="red" )
# ax.plot([O2_in_ground[0], I2_in_ground[0]],[O2_in_ground[1], I2_in_ground[1]],[O2_in_ground[2], I2_in_ground[2]], c="blue" )

ax.set_xlim(-2,2)
ax.set_ylim(-2,2)
ax.set_zlim(-2,2)

(-2.0, 2.0)

In [9]:
lever_arms_Q1 = []
lever_arms_Q2 = []
new_origin_vec = Vec3(*new_origin_test)
new_insertion_vec = Vec3(*new_insertion_test)
for i, c in enumerate(model.getCoordinateSet()):
    # c: osim.Coordinate = c
    FJ: osim.Joint = c.getJoint().getParentFrame()
    origin_in_J_Q1 = B_origin.findStationLocationInAnotherFrame(Q1, new_origin_vec, FJ).to_numpy()
    insertion_in_J_Q1 = B_insertion.findStationLocationInAnotherFrame(Q1, new_insertion_vec, FJ).to_numpy()
    la = np.cross(insertion_in_J_Q1, origin_in_J_Q1) / np.linalg.norm(insertion_in_J_Q1 - origin_in_J_Q1)
    w_Q1 = omegas_in_joints_Q1[:, i]
    la = w_Q1 @ la
    lever_arms_Q1.append(la)

    origin_in_J_Q2 = B_origin.findStationLocationInAnotherFrame(Q2, new_origin_vec, FJ).to_numpy()
    insertion_in_J_Q2 = B_insertion.findStationLocationInAnotherFrame(Q2, new_insertion_vec, FJ).to_numpy()

    la = np.cross(insertion_in_J_Q2, origin_in_J_Q2) / np.linalg.norm(insertion_in_J_Q2 - origin_in_J_Q2)
    w_Q2 = omegas_in_joints_Q2[:, i]
    la = w_Q2 @ la
    lever_arms_Q2.append(la)

lever_arms_Q1 = np.array(lever_arms_Q1)
lever_arms_Q2 = np.array(lever_arms_Q2)


la_by_derivatives_Q1 = common.lever_arm_matrix(Q1, [muscle], model.getCoordinateSet())
la_by_derivatives_Q2 = common.lever_arm_matrix(Q2, [muscle], model.getCoordinateSet())

print(lever_arms_Q1.round(4).tolist())
print(la_by_derivatives_Q1.ravel().round(4).tolist())

print((lever_arms_Q1[:3] / np.linalg.norm(lever_arms_Q1[:3])).round(4).tolist())
print((la_by_derivatives_Q1[:3] / np.linalg.norm(la_by_derivatives_Q1[:3])).ravel().round(4).tolist())

print(lever_arms_Q2.round(4).tolist())
print(la_by_derivatives_Q2.ravel().round(4).tolist())

print((lever_arms_Q2[:3] / np.linalg.norm(lever_arms_Q2[:3])).round(4).tolist())
print((la_by_derivatives_Q2[:3] / np.linalg.norm(la_by_derivatives_Q2[:3])).ravel().round(4).tolist())


[0.0722, -0.0608, -0.2138, -0.0893, 0.3872, 1.2977, -0.2051, 0.0012, -0.7951, 0.685, -0.4002, -0.1965]
[0.0706, -0.0596, -0.2092, -0.0854, 0.3853, 1.3011, -0.2036, -0.0042, -0.7931, 0.68, -0.3989, -0.1945]
[0.3088, -0.2604, -0.9148]
[0.3088, -0.2604, -0.9148]
[-0.1916, -0.0005, -0.163, 0.0535, -0.6136, -0.7816, 0.1209, 0.446, 1.332, -0.6102, 0.6466, 1.0553]
[-0.1876, -0.0005, -0.1596, 0.0515, -0.6161, -0.7833, 0.1244, 0.4488, 1.3303, -0.6104, 0.6434, 1.0534]
[-0.7617, -0.0022, -0.648]
[-0.7617, -0.0022, -0.648]


In [118]:
# LA = g3c_tools.random_conformal_point()^g3c_tools.random_conformal_point()^ni
# LA

-(84.0107^e124) - (84.0107^e125) - (18.00065^e134) - (18.00065^e135) + (7.5526^e145) - (68.79159^e234) - (68.79159^e235) - (27.31799^e245) - (12.03772^e345)

In [555]:
def null_rotor():
    return (0.0 * Px).exp()

def euler_extrinsic(rx, ry, rz):
    return (-0.5 * rx * Px).exp() * (-0.5 * ry * Py).exp() * (-0.5 * rz * Pz).exp()

def euler_intrinsic(rx, ry, rz):
    return  (-0.5 * rz * Pz).exp() * (-0.5 * ry * Py).exp() * (-(rx*Px) / 2).exp()

def translator(tx, ty, tz):
    return (-(tx*(e1^ni) + ty*(e2^ni) + tz*(e3^ni))/2).exp()

X = up(3*e1 + e2 + e3)

toG = null_rotor()
toB1 = null_rotor()
toJ1 = translator(*t1) * euler_extrinsic(*r1)
toJ1_child = euler_extrinsic(*q1)
toB2 = (translator(*t2) * euler_extrinsic(*r2)).inv()
toJ2 = translator(*t3) * euler_extrinsic(*r3)
toJ2_child = euler_extrinsic(*q2)
toB3 = (translator(*t4) * euler_extrinsic(*r4)).inv()

R = toG * toB1 * toJ1 * toJ1_child * toB2 * toJ2 * toJ2_child * toB3
print(down(R * X * ~R))

R = ~(toJ1_child * toB2)
print(down(R * X * ~R))



-(3.65384^e1) - (1.47323^e2) - (3.80405^e3)
-(3.53275^e1) - (0.62826^e2) + (0.63153^e3)


In [556]:
X = [1,2,3]
J_osim_dir = common.station_jacobian(model, state, X, B3)
print(J_osim_dir)
J_osim_moment = common.frame_jacobian(model, state, B3)
print(J_osim_moment)

[[ 4.81445088  0.38097219 -5.40392184 -3.91008054  0.19380367 -1.61455302]
 [-2.55270241 -1.60491412  0.52423453  1.72284215  1.14322941 -2.32923028]
 [ 0.59081906  1.33186891  1.48128821 -0.63139483 -1.23297924  2.86814419]]
[[-0.450548   -0.11785585 -0.09717794  0.34988903  0.56250767 -0.75490528]
 [-0.73119138  0.61744313  0.7722644   0.50490582  0.56033015  0.64796912]
 [ 0.51221642  0.77773645 -0.62782493 -0.78908034  0.60795991  0.10126223]]


In [572]:
EE = up(e1 + 2*e2 + 3*e3)
o = no # center of J1

Base = np.array([e1, e2, e3])

# Describing the origin in J1
R = ~(toG * toB1 * toJ1)
o_in_J1 = R * no * ~R
# o_in_J1 = no
# o_in_J1 = up(e1)
print(point_to_np(o_in_J1))
print(ground.findStationLocationInAnotherFrame(state, Vec3(0), J1.getParentFrame()).to_numpy())
print()
EE_in_origin = (B3.findStationLocationInGround(state, Vec3(1,2,3)).to_numpy())


R =  toJ1_child * toB2 * toJ2 * toJ2_child * toB3
EE_in_J1 = R * EE * ~R
print(down(EE_in_J1))
print(B3.findStationLocationInAnotherFrame(state, Vec3(1,2,3), J1.getParentFrame()).to_numpy())

# PLA = EE_in_J1^(e1 - down(EE_in_J1))^ni
PLA = (no^down(EE_in_J1)^(e1)^ni) # Plane centered at no works ?
PLA = (down(EE_in_J1)^(e1)^ni) # Direction works
PLA = (o_in_J1^down(EE_in_J1)^(e1)^ni) # Plane centered at joint works
# PLA = o_in_J1 ^ EE_in_J1 ^ up(e1)
PLA = (o_in_J1^e1^ni) + (down(EE_in_J1) ^ e1 ^ ni)
PLA = (o_in_J1 + down(EE_in_J1)) ^ e1 ^ ni # Line = sphere^dir^ni
PLA = up(down(o_in_J1) + down(EE_in_J1)) ^ e1 ^ ni # Line = sphere^dir^ni
# PLA = (EE_in_J1 ^ o_in_J1 ^ e1) ^ ni
R = toG * toB1 * toJ1
print()
print("PLA", PLA)
PLA_in_ground = R * PLA * ~R
print("PLA in G", PLA_in_ground.dual())
print("PLA in G", PLA_in_ground)
print("PLA in G", PLA_in_ground.normal())


[0.8617347  0.0625609  0.17393678]
[0.8617347  0.0625609  0.17393678]

(1.28791^e1) + (2.7502^e2) - (4.74138^e3)
[ 1.28790986  2.75020029 -4.74138085]

PLA -(2.81276^e124) - (2.81276^e125) + (4.56744^e134) + (4.56744^e135) + (1.0^e145)
PLA in G -(0.51222^e12) - (0.73119^e13) + (4.81445^e14) + (4.81445^e15) + (0.45055^e23) - (2.5527^e24) - (2.5527^e25) + (0.59082^e34) + (0.59082^e35)
PLA in G -(0.59082^e124) - (0.59082^e125) - (2.5527^e134) - (2.5527^e135) - (0.45055^e145) - (4.81445^e234) - (4.81445^e235) - (0.73119^e245) + (0.51222^e345)
PLA in G -(0.59082^e124) - (0.59082^e125) - (2.5527^e134) - (2.5527^e135) - (0.45055^e145) - (4.81445^e234) - (4.81445^e235) - (0.73119^e245) + (0.51222^e345)


In [541]:
OK = (o_in_J1 + down(EE_in_J1)) ^ e1
K = R * OK * ~R
print(K)
print((K^ni)^(R*e1*~R))

(2.14078^e12) - (0.70375^e13) + (0.46193^e14) + (0.30135^e15) - (4.07145^e23) - (1.58795^e24) - (2.57479^e25) + (1.40053^e34) + (1.41954^e35) - (0.33205^e45)


In [530]:
o_in_J1 + down(EE_in_J1)

(0.01505^e1) - (4.1251^e2) + (1.57185^e3) + (0.49072^e4) + (1.49072^e5)

In [536]:
print(down(EE_in_J1))
print(down(o_in_J1))
down(o_in_J1) + down(EE_in_J1)

(0.3471^e1) - (3.66956^e2) + (2.86169^e3)
-(0.33205^e1) - (0.45553^e2) - (1.28984^e3)


(0.01505^e1) - (4.1251^e2) + (1.57185^e3)

In [413]:
o_in_J1 = ground.findStationLocationInAnotherFrame(state, Vec3(0), J1.getParentFrame()).to_numpy()
EE_in_J1 = B3.findStationLocationInAnotherFrame(state, Vec3(1,2,3), J1.getParentFrame()).to_numpy()

Base = np.array([e1, e2, e3])
L = up(o_in_J1@Base) ^ up(EE_in_J1@Base) ^ ni
print(L)

omega = np.cross(o_in_J1, EE_in_J1)
print(omega)

-(5.9099^e124) - (5.9099^e125) + (2.96217^e134) + (2.96217^e135) + (2.9805^e145) + (2.04595^e234) + (2.04595^e235) - (3.01239^e245) + (2.5417^e345)
[ 2.0459544  -2.96216906 -5.90990259]
